# Módulo 1: Análisis de datos en el ecosistema Python

### Sesión (25)

# Modelado estadístico

El modelado estadístico (**Statistical Modeling**) en series temporales se refiere a la aplicación de **`métodos estadísticos`** para analizar y modelar los datos secuenciales.

El modelado estadístico de series temporales implica **seleccionar** un modelo apropiado en función de las características de los datos, **ajustar** el modelo a los datos y **utilizar** el modelo para hacer **predicciones** o probar hipótesis.

![Time-series-modeling.png](attachment:Time-series-modeling.png)

In [ ]:
# importamos las librerías necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# Modificamos los parámetros de los gráficos en matplotlib
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 12, 6 # el primer dígito es el ancho y el segundo el alto
rcParams["font.weight"] = "bold"
rcParams["font.size"] = 10
rcParams["axes.labelweight"] = "bold"

### Airline Passenger Dataset

Importamos los datos del ejemplo disponible en la librería ___seaborn___ que contiene el número total de pasajeros aéreos de forma mensual.

In [ ]:
import seaborn as sns
import pandas as pd

# Cargar el dataset de "flights"
df_air = sns.load_dataset('flights')
df_air

In [ ]:
# Consultamos la información del dataset descargado
df_air.info()

In [ ]:
# Crear una nueva columna con la unificación de otras dos
df_air['year_month'] = df_air.apply(lambda x: str(x['year']) + '-' + x['month'], axis=1)

# Convertir la columna en fechas tipo DatetimeIndex
df_air['fechas'] = pd.to_datetime(df_air['year_month'], format='%Y-%b')

# Convertir la columna de fechas a los índices del DataFrame e indicar que los datos son "mensuales"
df_air.set_index('fechas', inplace=True)
df_air.index.freq = 'MS'

# Quitar las columnas no necesarias
df_air.drop(columns=['year', 'month', 'year_month'], inplace=True)

df_air

In [ ]:
# Visualizar el DataFrame creado con los datos de la serie temporal
plt.plot(df_air)
plt.show()

In [ ]:
# Gráfica interactiva con Plotly
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_air.index,
    y=df_air['passengers'],
    mode="markers+lines"
    ))

fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=26,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

## Time Series Forecasting

#### Preparación de conjuntos de entrenamiento y prueba:

En primer lugar debemos crear dos conjuntos diferentes para poder entrenar el modelo con uno y poder analizar los resultados del modelo en un periodo de prueba, del modo que para el modelo cuente como _"futuro"_.

El tamaño mínimo del conjunto de test tiene que cubrir el horizonte de predicción. Procedemos a crear modelos que sean capaces de predecir los próximos 12 meses.

- **Test**: El último año
- **Training**: Todo el histórico salvo el último año

![traintest-1.png](attachment:traintest-1.png)

In [ ]:
# Definir el periodo de prueba (horizonte de predicción)
horizonte = 12  # La cantidad de puntos a predecir
df_test = df_air.tail(horizonte)
df_test

In [ ]:
# Filtrar la serie original para sacar el periodo de entrenamiento
df_train = df_air[df_air.index.isin(df_test.index)==False]
df_train

In [ ]:
# Comprobar las dimensiones de los conjuntos de datos creados
print("tamaño del dataset: ", len(df_air))
print("tamaño de los datos de entrenamiento: ", len(df_train))
print("tamaño de los datos de test: ", len(df_test))

Visualizamos de forma interactiva los periodos marcados como entrenamiento y prueba.

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=df_train['passengers'],
    name="Entrenamiento",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test['passengers'],
    name="Test",
    mode="markers+lines"
    ))


fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=22,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

### **Benchmark Models**

En el proceso de modelado, los modelos de referencia (**benchmarks**) son modelos simples y fáciles de implementar que sirven como referencia para **evaluar el rendimiento de modelos más complejos**.

Los modelos de referencia se utilizan a menudo para **establecer un nivel mínimo de rendimiento** que debe superar un modelo más complejo para que se considere útil.

### Average Model

En este primer modelo básici, las predicciones de todos los valores futuros son iguales al _promedio_ (**"media"**) de los datos históricos.

In [ ]:
# Calcular la media de los valores históricos
df_train.mean()

In [ ]:
# Crear predicciones iguales al promedio
media = df_train.mean().values[0]
pred_avg = pd.DataFrame(data=[media]*df_test.size, index=df_test.index, columns=['pred'])
pred_avg

In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_avg, label='Predicción - Average')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

Creamos la gráfica de _plotly_, donde claramente se ve que este modelo tiene una caída considerable en las predicciones de la serie temporal.

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=df_train['passengers'],
    name="Entrenamiento",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test['passengers'],
    name="Test",
    mode="lines"
    ))


fig.add_trace(go.Scatter(
    x=pred_avg.index,
    y=pred_avg['pred'],
    name="Predicción (Average Model)",
    mode="markers+lines"
    ))



fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=22,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

### Naïve model (**Random Walk**)


El método **Naïve** o de **persistencia**, simplemente considera que todas las predicciones son iguales que al valor de la **última observación**. Este método funciona **notablemente bien** para muchas series temporales `económicas` y `financieras`.

Debido a que un pronóstico ingenuo es óptimo cuando los datos siguen un recorrido aleatorio, también se denominan como ***random walk***.

In [ ]:
# Sacar el último valor de la serie de entrenamiento
df_train.tail(1)

In [ ]:
# Crear las predicciones para el modelo "random walk"
ultimo = df_train.tail(1).values[0]
pred_rw = pd.DataFrame(data=[ultimo]*df_test.size, index=df_test.index, columns=['pred'])
pred_rw

In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_rw, label='Predicción - Naive (Random Walk)')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

Podemos observar que respecto al modelo anterior, al menos no presenciamos una caída en el nivel de los valores predichos.

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=df_train['passengers'],
    name="Entrenamiento",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test['passengers'],
    name="Test",
    mode="lines"
    ))


fig.add_trace(go.Scatter(
    x=pred_rw.index,
    y=pred_rw['pred'],
    name="Predicción (Random Walk)",
    mode="markers+lines"
    ))



fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=22,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

### Drift Model

Una variación del método ingenuo (_random walk_) es permitir que los valores a predecir aumenten o disminuyan con el tiempo, donde la cantidad de cambio con el tiempo se establece como el **cambio promedio observado** en los datos históricos.

![drift.JPG](attachment:drift.JPG)



Esto es equivalente a trazar **una línea entre la primera y la última observación** y extrapolarla al futuro.

In [ ]:
# Calcular las predicciones del método "random walk with drift"
pendiente = (df_train.values[-1] - df_train.values[0]) / (len(df_train) - 1)
datos_drift = [df_train.values[-1] + pendiente*x for x in range(1, horizonte+1)]
pred_drift = pd.DataFrame(data=datos_drift, index=df_test.index, columns=['pred'])
pred_drift

In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_drift, label='Predicción - Drift method')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

Se ve que el nivel de los valores predichos aumenta a lo largo del tiempo, de acuerdo al cambio incremental en el nivel de la serie temporal.

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=df_train['passengers'],
    name="Entrenamiento",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test['passengers'],
    name="Test",
    mode="lines"
    ))


fig.add_trace(go.Scatter(
    x=pred_drift.index,
    y=pred_drift['pred'],
    name="Predicción (Drift)",
    mode="markers+lines"
    ))


fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=22,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

### Seasonal Naïve model

Un método útil para datos altamente estacionales consiste en que cada predicción sea igual al último valor observado de la misma temporada, por ejemplo el mismo mes del año anterior.

In [ ]:
# Sacar el último periodo (el último año) de la serie de entrenamiento
df_train.tail(12)

In [ ]:
pred_sn = pd.DataFrame(data=df_train.tail(12).values, index=df_test.index, columns=['pred'])
pred_sn

In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_sn, label='Predicción - Seasonal Naïve')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

Se puede apreciar que **la replica del año anterior** sigue mucho mejor la evolución de los valores reales. Sin embargo, claramente existe un **sesgo en la predicción** por no contar con el aumento en el nivel de los pasajeros en egeneral de acuerdo a la **tendencia global** de la serie temporal.  

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=df_train['passengers'],
    name="Entrenamiento",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test['passengers'],
    name="Test",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=pred_sn.index,
    y=pred_sn['pred'],
    name="Predicción (Seasonal Naive)",
    mode="markers+lines"
    ))

fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=22,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

### **In-Sample** Model Evaluation

La evaluación dentro de la muestra (_In-sample evaluation_) consiste en **evaluar las estimaciones del modelo** para el mismo conjunto de **datos utilizados en el proceso de entrenamiento**. Si bien esta evaluación proporciona información sobre qué tan bien se ajusta el modelo a los datos disponibles, **no ofrece una información** respecto a la **capacidad de generalización del modelo** a los datos nuevos y no vistos anteriormente.

Algunas medidas comunes utilizadas para evaluar el ajuste dentro de la muestra de un modelo de serie temporal incluyen:

- **Residuales**: Los errores de predicción en el conjunto de entrenamiento son loa valores residuales del modelo que deben estar **en torno a cero** y **no deben exhibir ningún patrón** concreto.

- **Calidad de ajuste (_Goodness-of-fit_)**: El coeficiente de **R-cuadrado**, el criterio de información de Akaike (**AIC**), el criterio de información bayesiano (**BIC**) son estadísticas que se pueden utilizar para **comparar el ajuste de diferentes modelos** y seleccionar el mejor modelo. Tanto _AIC_ como _BIC_ son medidas de la **compensación entre la calidad de ajuste del modelo y la complejidad** del mismo. Cuanto **menor** sea la puntuación _AIC_ o _BIC_, **mejor** se ajustará el modelo a los datos.

La evaluación dentro del periodo de entrenamiento **no debe usarse como el único criterio** para seleccionar un modelo de serie temporal. El sobreajuste o el ___Overfitting___ puede ocurrir cuando un modelo es **demasiado complejo** y se ajusta demasiado bien a los datos disponibles, lo que da como resultado **un rendimiento muy pobre de generalización** en datos nuevos. Por lo tanto, la evaluación fuera de la muestra (_Out-of-sample evaluation_) es esencial para valorar la calidad de los modelos de series temporales.

![fifth-order-overfit.png](attachment:fifth-order-overfit.png)

#### Forecasing accuracy

Hay diferentes métricas para **evaluar las predicciones generadas** por un modelo de series temporales **al igual que los modelos de regresión**. La mayoría de estas medidas se basan en la **similitud de los valores predichos con reales**.

**El error de un modelo de series temporales** es la diferencia entre los datos reales y las predicciones del modelo. El error puede ser determinado de multiples maneras:

- Error medio absoluto (**MAE**) es la media del valor absoluto de los errores.
- Error cuadrático medio (**MSE**) es la media de los errores al cuadrado.
- Error medio absoluto porcentual (**MAPE**) es la media del porcentaje de errores absoluto. Nos hace una idea sobre el porcentaje de error, aunque no funciona tan bien si tenemos datos cerca de cero.
- Raiz cuadrada del error cuadrático medio (**RMSE**) es la raíz cuadrada de la anterior medida.

![formula-MAE-MSE-RMSE-RSquared.JPG](attachment:formula-MAE-MSE-RMSE-RSquared.JPG)

**R-cuadrado** (también se conoce como **coeficiente de determinación**) no es un error, si no, una métrica popular para la precisión del modelo. Determina la **`capacidad de un modelo para predecir futuros resultados`**, es decir, representa como de cerca a realidad están los valores de los datos de la línea de regresión ajustada.

El mejor resultado posible es 1.0, y ocurre cuando la predicción coincide con los valores de la variable objetivo. $R^{2}$ puede tomar valores negativos pues la predicción puede ser arbitrariamente mala. Cuanto **más alto sea el $R^{2}$, mejor** encaja el modelo a los datos.

Ahora consultamos las métricas de calidad de cada modelo sobre el periodo de prueba, del mismo modo de modelos de **regresión** mediante la librería _sklearn_:

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Métricas de evaluación del modelo
print("Out-of-sample performance:  Average Model")
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_avg))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_avg)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_avg))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_avg)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_avg))

La librería _statsmodels_ también dispone de métodos propios para sacar algunas de estas métricas:

In [ ]:
from statsmodels.tools.eval_measures import meanabs
from statsmodels.tools.eval_measures import mse
from statsmodels.tools.eval_measures import rmse

print("Out-of-sample performance:  Average Model")
print('Mean Absolute Error (MAE) by (statsmodels):', meanabs(df_test, pred_avg))
print('Mean Squared Error (MSE) by (statsmodels):', mse(df_test, pred_avg))
print('Root Mean Squared Error (RMSE) by (statsmodels):', rmse(df_test, pred_avg))

In [ ]:
# Métricas de evaluación del modelo
print("Out-of-sample performance:  Random Walk (persistencia)")
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_rw))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_rw)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_rw))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_rw)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_rw))

In [ ]:
# Métricas de evaluación del modelo
print("Out-of-sample performance:  Drift method")
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_drift))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_drift)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_drift))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_drift)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_drift))

In [ ]:
# Métricas de evaluación del modelo
print("Out-of-sample performance:  Seasonal Naive")
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_sn))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_sn)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_sn))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_sn)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_sn))

**Benchmark results**

Como podemos observar estos modelos de referncia generan un rendimiento de base que por ser superado por otros modelos pueden indicar la utilidad de usarlos. Graficamos el error absoluto medio en la predicción del último año con estos modelos de Benchmarks:

In [ ]:
df_metricas = pd.DataFrame({"Modelo": ['Average', 'Random Walk', 'Drift', 'Seasonal Naive'],
                            "MAE": [213.67, 76.0, 66.31, 47.83]})

sns.lineplot(data=df_metricas, x=df_metricas['Modelo'], y=df_metricas['MAE'], marker='s')
plt.ylim(20,220)
plt.show()

---

### Modelos de Autoregresión (AR)

Los modelos tipo AR (**Autoregressive Models**), son un enfoque popular para la previsión de series temporales que se basa en la idea de que **los valores pasados de una serie temporal se pueden utilizar para predecir sus valores futuros**. Es decir, el valor un paso de tiempo dado se modela como una **combinación lineal de sus valores pasados**, y los coeficientes de la combinación lineal se estiman a partir de los datos disponibles en el periodo de entrenamiento.

El método **`AutoReg`** tiene un parámetro como el orden de la autorregresión (___p___) que determina el **número de valores pasados** de la serie temporal que se utilizan para predecir el valor actual.

Este se puede escribir como:

$y_t = c + φ_1y(t-1) + φ_2y(t-2) + … + φ_p*y(t-p) + ε_t$

donde $y_t$ es el valor de la serie temporal en el tiempo $t$, $c$ es un término constante, $φ_1, φ_2, …, φ_p$ son los coeficientes autorregresivos y $ε_t$ es un término de error de ruido blanco. Estos coeficientes autorregresivos se estiman mediante el método de máxima verosimilitud (_Maximum Likelihood_) o regresión de mínimos cuadrados (_OLS_).

Como sabemos que nuestros datos tienen una dependencia a los valores pasados generamos un modelo ___AR(12)___

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

modelo_ar = AutoReg(df_train, lags=12)
modelo_ar

In [ ]:
# Ajustar el modelo a los datos de entrenamiento
fit_ar = modelo_ar.fit()
fit_ar


El método `.summary()` en muchos modelos de la librería `statsmodels` proporciona un resumen de los resultados y la calidad del modelo ajustado:

- **Modelo**: Esta sección proporciona una breve descripción del modelo, incluida la variable dependiente, el método utilizado para ajustar el modelo y el número de observaciones y parámetros.

- **Criterios de información**: esta sección proporciona varios criterios de información, como el criterio de información de Akaike (_AIC_) y el criterio de información bayesiano (_BIC_), que se pueden utilizar para comparar el ajuste de diferentes modelos.

- **Coeficientes**: esta sección enumera los coeficientes estimados para cada variable de retraso (_lags_).

In [ ]:
fit_ar.summary()

#### Fitted-Model Evaluation

Ahora podemos analizar hasta qué punto se ha ajustado este modelo a los datos de entrenamiento (___In-sample model evaluation___). Básicamente calculamos la estimación del modelo para cada observación en este periodo como la predicción de un paso adelante (___one-step-ahead forecast___)

Podemos extraer los valores ajustados o las estimaciones del modelo de serie temporal mediante el atributo `fittedvalues`

In [ ]:
# Consultamos los valores ajustados en el periodo de entrenamiento
estim_ar = fit_ar.fittedvalues
estim_ar

In [ ]:
plt.plot(df_train, label='Entrenamiento')
plt.plot(estim_ar, label='Estimación AR(12)')
plt.title("Datos reales vs. Estimación del modelo (In-Sample forecasting)")
plt.legend()
plt.show()

Vemos que existe una diferencia entre el periodo de entrenamiento y el de valores estimados por el modelo:

In [ ]:
print("La diferencia del tamaño de datos = ", len(df_train) - len(estim_ar))

Esto pasa por la necesidad del modelo de **disponer de un conjunto mínimo de muestras** para poder generar una predicción. Otra forma de recuperar los valores ajustados por el modelo sería sacar la predicción por defecto usando el método `.predict()` que nos devuelve un **objeto con el mimo tamaño** que los datos de entrenamiento:

In [ ]:
estim_ar_train = fit_ar.predict()
estim_ar_train

In [ ]:
# Comparar los valores reales con la estimación del modelo
sns.scatterplot(x=df_train.values.flatten(), y=estim_ar_train)
plt.plot([100,600], [100,600], color='r', linestyle=':')
plt.title("Valores reales vs. valores estimados (In-Sample forecasting) - AR(12)")
plt.show()

 Vamos a analizar los errores en el periodo de entrenamiento.

In [ ]:
erorr_ar_train = pd.Series(data=df_train.values.flatten()-estim_ar_train.values.flatten(),
                           index=df_train.index)
erorr_ar_train

Los errores dentro del periodo de entrenamiento, también se consideran como **valores residuales** que corresponden a la parte no capturada por el modelo. Un aspecto importante para evaluar el modelo ajustado a los datos de entrenamiento consiste en analizar las características de estos valores como el **componente residual** que idealmente serían como un **ruido blanco**:

- **`Media cero`**
- **`Varianza constante`**
- **`Autocorrelación cero`**
- **`Distribución normal`**

In [ ]:
# Se pueden extraer directamente desde el modelo entrenado mediante el atributo "resid"
resid_ar = fit_ar.resid
resid_ar

In [ ]:
# Las estadísticas del componente residual
resid_ar.describe().round(3)

In [ ]:
plt.plot(resid_ar)
plt.title("Componente residual del modelo AR (In-Sample fitted errores)")
plt.axhline(y=0, color='r', linestyle=':')
plt.show()

In [ ]:
# El histograma del componente residual (in-sample-errors)
sns.histplot(data=resid_ar, bins=50)
plt.show()

Vemos que los residuos tienen **la media cero**, nos fijamos en la varianza de estos valores.

In [ ]:
win = 20
resid_ar_std = resid_ar.rolling(win).std().iloc[win-1::win]
plt.plot(resid_ar_std, label='Desviación estándar')
plt.axhline(y=9.6, color='r', linestyle='--')
plt.title("Características estadísticas: Residual - AR(12)")
plt.ylim(0,50)
plt.legend()
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(resid_ar, lags=37)
plt.xticks(np.arange(37))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

plot_pacf(resid_ar, lags=37, method='ywm')
plt.xticks(np.arange(37))
plt.ylim(-1.1,1.1)
plt.show()

Además de revisar las características estadísticas del componente residual, debemos asegurarnos de que **no haya ningún patrón** entre los errores y los valores de la serie o los valores estimados.

In [ ]:
# Coeficiente de correlación entre valores reales y los errores
df_train['passengers'].corr(erorr_ar_train).round(4)

In [ ]:
sns.scatterplot(x=df_train.values.flatten(), y=erorr_ar_train)
plt.title("Valores reales versus valores residuales - AR(12)")
plt.show()

In [ ]:
# Coeficiente de correlación entre valores estimados y los errores
estim_ar.corr(resid_ar).round(6)

In [ ]:
sns.scatterplot(x=estim_ar, y=resid_ar)
plt.title("Valores estimados versus valores residuales - AR(12)")
plt.show()

De forma resumida podemos investigar la calidad del modelo entrenado (_fitted model_) mediante varias gráficas que se pueden sacar por defecto mediante el método `plot_diagnostics()`

Estas curvas se pueden utilizar para evaluar la calidad del ajuste del modelo a los datos y para identificar cualquier problema con el modelo. Esto es lo que representa cada curva:

- **Residuos estandarizados**: esta gráfica muestra los residuos estandarizados del modelo. Si el modelo se ajusta bien a los datos, los residuos deben **estar dispersos aleatoriamente alrededor de 0**, sin ninguna desviación obvia, patrones o tendencias.

- **Gráfica de Histograma y Densidad**: Esta gráfica muestra la distribución de los residuos estandarizados. Si los residuos se distribuyen normalmente, deben **seguir una curva en forma de campana** en el histograma. El gráfico de densidad muestra una versión suavizada del histograma. Si los residuos se distribuyen normalmente, la gráfica de densidad debe ser aproximadamente simétrica alrededor de 0.

- **Gráfica Q-Q normal**: esta gráfica muestra los cuantiles de los residuos estandarizados frente a los cuantiles de una distribución normal. Si los residuos se distribuyen normalmente, los puntos deben **seguir una línea recta**. Cualquier desviación de una línea recta indica no normalidad en los residuos.

- **Correlograma**: este gráfico muestra la autocorrelación de los residuos estandarizados en diferentes retrasos o _lags_. Si el modelo se ajusta bien a los datos, los residuos **no deberían mostrar ninguna autocorrelación significativa** en ningún retraso.

In [ ]:
# Sacar las gráficas del rendimiento del modelo ajustado
fit_ar.plot_diagnostics()
plt.show()

#### Prediction evaluationEvaluating (_forecasting Acuuracy_)

Después de analizar la calidad del modelo ajustado a los datos de entrenamiento, nos debemos fijar en el rendimiento del modelo a la hora de predecir los valores no vistos que serían las estimaciones al futuro  (***Out-of-Sample model evaluation***)

In [ ]:
# Calcular las predicciones con los valores estimados por el modelo a futuro
fit_ar.predict(start=df_test.index[0],
               end=df_test.index[-1])

Podemos indicar el **hoizonte** de la predicción como el número de los valores a predecir por el modelo

In [ ]:
pred_ar = fit_ar.forecast(steps=horizonte)
pred_ar


Redondeamos los valores predichos para tenerlos como número de pasajeros

In [ ]:
pred_ar = pred_ar.round()

In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_ar, label='Predicción - AR(12)')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=df_train['passengers'],
    name="Entrenamiento",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test['passengers'],
    name="Test",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=pred_sn.index,
    y=pred_sn['pred'],
    name="Predicción (Seasonal Naive)",
    mode="markers+lines"
    ))

fig.add_trace(go.Scatter(
    x=pred_ar.index,
    y=pred_ar,
    name="Predicción (AR)",
    mode="markers+lines"
    ))

fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=22,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

Se puede ver que las predicciones de este modelo, además de **seguir la forma del periodo estacional**, se adaptan a los **niveles crecientes** que son presentes en los datos reales.

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_ar))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_ar)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_ar))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_ar)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_ar))

Como lo habíamos intuido mvisualmente, este modelo está generando **predicciones más precisos** que los modelos de referncia. Ahora nos fijamos en el **nivel de incertidumbre** que tenemos en los valores predichos.

#### Intervalos de confianza (___Confidence intervals___)

En la predicción de series temporales, los intervalos de predicción o intervalos de confianza nos dan una idea de **qué tan inciertos son nuestras predicciones**. Los intervalos de confianza para los valores predichos representan **un rango de valores** dentro del cual esperamos que se encuentren los valores reales con **un cierto nivel de confianza**. Un intervalo de **confianza del 95 %** significa que **hay una probabilidad del 95 % de que el valor verdadero se encuentre dentro de este intervalo**.

Usamos el método `conf_int()` sobre los datos extraídos como la predicción del modelo mediante la función `get_prediction` para el periodo en cuestión. Los intervalos por defecto marcan **un nivel de 95% de confianza** considerando `alpha=0.05` que se pueden modificar si así se desea.

In [ ]:
conf_ar = fit_ar.get_prediction(start=df_test.index[0], end=df_test.index[-1]).conf_int().round()
conf_ar

Analizamos la evolución de los rangos que marcan los intervalos de confianza

In [ ]:
conf_ar['delta'] = conf_ar.apply(lambda x: x['upper'] - x['lower'], axis=1)
conf_ar

In [ ]:
plt.plot(conf_ar['delta'])
plt.title("Evolución del rango de los intervalos de confianza  - AR(12)")
plt.show()

Como es de esperar, el rango de los intervalos **se aumenta con el tiempo** porque **se van acumulando los errores y los incertidumbres** presentes en las predicciones del modelo por ser de tipo `One-step-ahead forecasts`.

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=df_train['passengers'],
    name="Entrenamiento",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test['passengers'],
    name="Test",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=pred_sn.index,
    y=pred_sn['pred'],
    name="Predicción (Seasonal Naive)",
    mode="markers+lines"
    ))

fig.add_trace(go.Scatter(
    x=pred_ar.index,
    y=pred_ar,
    name="Predicción (AR)",
    mode="markers+lines"
    ))

fig.add_trace(go.Scatter(
    x=conf_ar.index,
    y=conf_ar['lower'],
    name="lower",
    mode="lines",
    line=dict(width=0),
    showlegend=False
    ))

fig.add_trace(go.Scatter(
    x=conf_ar.index,
    y=conf_ar['upper'],
    name="upper",
    mode="lines",
    line=dict(width=0),
    showlegend=False,
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty'
    ))

fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=22,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

---

### **ARIMA** (Autoregressive Integrated Moving Average)

_ARIMA_ (**promedio móvil integrado autorregresivo**) es un método estadístico ampliamente utilizado para modelar las series temporales. Los modelos ARIMA son una combinación de tres métodos separados: autoregresión (___AR___), integración (___I___) y media móvil (___MA___). El modelo ARIMA está especificado por tres parámetros ___(p, d, q)___ que determinan el número de términos autoregresivos, diferenciadores y de media móvil, respectivamente:

- **La autorregresión (AR)**: Ees un método que utiliza observaciones pasadas para predecir valores futuros. El orden del modelo _AR(p)_ especifica cuántos valores pasados de la serie temporal se utilizan para predecir el valor actual.

- **La integración (I)**: Es un método que ajusta la serie temporal para hacerla estacionaria, lo que significa que sus propiedades estadísticas se mantienen constantes en el tiempo. El orden de diferenciación _(d)_ especifica el número de veces que se diferencia la serie de tiempo para hacerla estacionaria.

- **La media móvil (MA)**: Es un método que utiliza errores pasados para predecir valores futuros, por defecto para el modelo de promedio de todas las observaciones. El orden del modelo _MA(q)_ especifica cuántos errores pasados se utilizan para predecir el valor actual.

El modelo ARIMA combina estos tres métodos para crear una poderosa herramienta para el pronóstico de series de tiempo. La forma general de un modelo ARIMA es:

$ARIMA(p, d, q) = AR(p) + I(d) + MA(q)$

![ARIMA.png](attachment:ARIMA.png)

Vamos a crear un modelo ARIMA por ejemplo con la combinación de _(12,1,1)_ para añadir una diferenciación del primer orden y un componente de _Moving Average_ del mismo orden.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
modelo_arima = ARIMA(df_train, order=(12,1,1))
modelo_arima

In [ ]:
# Ajustar el modelo a los datos de entrenamiento
fit_arima = modelo_arima.fit()
fit_arima


In [ ]:
fit_arima.summary()

In [ ]:
# Consultamos los valores ajustados en el periodo de entrenamiento
estim_arima = fit_arima.fittedvalues
estim_arima

In [ ]:
plt.plot(df_train, label='Entrenamiento')
plt.plot(estim_arima, label='Estimación ARIMA(12,1,1)')
plt.title("Datos reales vs. Estimación del modelo (In-Sample forecasting)")
plt.legend()
plt.show()

In [ ]:
# Comparar los valores reales con la estimación del modelo
sns.scatterplot(x=df_train.values.flatten(), y=estim_arima)
plt.plot([0,600], [0,600], color='r', linestyle=':')
plt.title("Valores reales vs. valors estimados (In-Sample forecasting) ARIMA(12,1,1)")
plt.show()

In [ ]:
# Analizamos el componente residual
# Sacar las gráficas del rendimiento del modelo ajustado
fit_arima.plot_diagnostics()
plt.show()

resid_arima = fit_arima.resid

# Coeficiente de correlación entre valores reales y los errores
print(df_train['passengers'].corr(resid_arima).round(4))

# Coeficiente de correlación entre valores estimados y los errores
print(estim_arima.corr(resid_arima).round(4))

Vemos que la autocorrelación de los valores residuales no es significativo. Esto indica que el modelo ajustado tiene más calidad que el anterior.

In [ ]:
sns.scatterplot(x=df_train.values.flatten(), y=resid_arima)
plt.title("Valores reales versus valores residuales - ARIMA(12,1,1)")
plt.show()

In [ ]:
sns.scatterplot(x=estim_arima, y=resid_arima)
plt.title("Valores estimados versus valores residuales - ARIMA(12,1,1)")
plt.show()

In [ ]:
pred_arima = fit_arima.forecast(steps=horizonte).round()
pred_arima

In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_arima, label='Predicción - ARIMA(12,1,1)')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_arima))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_arima)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_arima))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_arima)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_arima))

Podemos apreciar que los resultados de este modelo **son algo mejores** que el modelo anterior de AR(12). Graficamos los valores predichos junto con los intervalos de confianza.

In [ ]:
# Calcular los intervalos de confianza
conf_arima = fit_arima.get_prediction(start=df_test.index[0], end=df_test.index[-1]).conf_int().round()

# Graficar los resultados
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=df_train['passengers'],
    name="Entrenamiento",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test['passengers'],
    name="Test",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=pred_ar.index,
    y=pred_ar,
    name="Predicción (AR(12))",
    mode="markers+lines"
    ))

fig.add_trace(go.Scatter(
    x=pred_arima.index,
    y=pred_arima,
    name="Predicción (ARIMA(12,1,1))",
    mode="markers+lines"
    ))

fig.add_trace(go.Scatter(
    x=conf_arima.index,
    y=conf_arima.iloc[:,0],
    name="lower",
    mode="lines",
    line=dict(width=0),
    showlegend=False
    ))

fig.add_trace(go.Scatter(
    x=conf_arima.index,
    y=conf_arima.iloc[:,1],
    name="upper",
    mode="lines",
    line=dict(width=0),
    showlegend=False,
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty'
    ))

fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=22,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

### Seasonal ARIMA (__SARIMA__)

Los modelos ARIMA también son capaces de modelar una amplia gama de datos estacionales, considerando **un componente especialmente pensado para el la estacionalidad**.

Un modelo **ARIMA estacional** o _SARIMA_ se forma al incluir términos estacionales adicionales en los modelos ARIMA que se especifica mediante cuatro parámetros:
- El período estacional (___m___)  
- La diferencia estacional (___D___)
- El orden autoregresivo estacional (___P___)
- El orden de promedio móvil estacional (___Q___).

El modelo SARIMA se puede escribir como $SARIMA(p,d,q)(P,D,Q)_m$:

![Sarima-seasonality.jpeg](attachment:Sarima-seasonality.jpeg)

La parte estacional del modelo consta de términos que son similares a los componentes no estacionales del modelo, pero implican **desplazamientos hacia atrás del período estacional**.

Vamos a comprobar el rendimiento de un modelo como $SARIMA(1,1,1)(1,0,1)12$ usando el método `SARIMAX` de la librería _statsmodels_

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
modelo = SARIMAX(df_train,
                order=(1,1,1),
                seasonal_order=(1,0,1,12))
fit_mod = modelo.fit()
print(fit_mod.summary().tables[0])
estim_mod = fit_mod.fittedvalues
pred_mod = fit_mod.forecast(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_mod))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_mod)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_mod))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_mod)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_mod))



Como cualquier otro modelo que dispone de hiperparámetros, estos modelos se pueden ajustar para **encontrar la combinación óptima de los parámetros** del modelo.

#### Auto ARIMA

Existen diferentes métodos para encontrar los ordenes óptimos de los modelos de ARIMA. Una herramienta común para automatizar esta búsqueda es el método `auto_arima` de la librería __pmdarima__.

`pmdarima` es paquete de Python que se basa en las bibliotecas _statsmodels_ y _scikit-learn_ y proporciona una interfaz conveniente para la selección de modelos, el ajuste de hiperparámetros y la evaluación de modelos de series temporales.

In [ ]:
pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima

fit_arima_auto = auto_arima(df_train,
                            start_p=1,
                            start_q=0,
                            max_p=20,
                            max_q=10,
                            seasonal=False,
                            stepwise=True)

In [ ]:
fit_arima_auto.summary()

In [ ]:
fit_arima_auto.fittedvalues()

Podemos usar el método `.predict_in_sample()` para sacar las estimaciones del modelo:

In [ ]:
fit_arima_auto.predict_in_sample()

In [ ]:
estim_arima_auto = fit_arima_auto.fittedvalues()
plt.plot(df_train, label='Entrenamiento')
plt.plot(estim_arima_auto, label='Estimación Auto_ARIMA')
plt.title("Datos reales vs. Estimación del modelo (In-Sample forecasting)")
plt.legend()
plt.show()

In [ ]:
pred_arima_auto = fit_arima_auto.predict(horizonte).round()
pred_arima_auto

In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_arima_auto, label='Predicción - Auto_ARIMA')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_arima_auto))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_arima_auto)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_arima_auto))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_arima_auto)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_arima_auto))

Vemos que esta búsqueda no nos lleva necesariamente a conseguir un modelo mejor que los anteriores. Probamos con los modelos tipo _SARIMA_ al incluir el componente estacional.

In [ ]:
from pmdarima.arima import auto_arima

fit_sarima_auto = auto_arima(df_train,
                             start_p=1,
                             start_q=1,
                             start_P=1,
                             D=1,
                             d=None,
                             m=12,
                             seasonal=True,
                             stepwise=True)

In [ ]:
fit_sarima_auto.summary()

In [ ]:
fit_sarima_auto.predict_in_sample()

In [ ]:
estim_sarima_auto = fit_sarima_auto.predict_in_sample()
plt.plot(df_train, label='Entrenamiento')
plt.plot(estim_sarima_auto, label='Estimación Auto_SARIMA')
plt.title("Datos reales vs. Estimación del modelo (In-Sample forecasting)")
plt.legend()
plt.show()

In [ ]:
pred_sarima_auto = fit_sarima_auto.predict(horizonte).round()
pred_sarima_auto

In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_sarima_auto, label='Predicción - Auto_SARIMA')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_sarima_auto))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_sarima_auto)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_sarima_auto))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_sarima_auto)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_sarima_auto))

Este modelo tiene un mejor rendimiento al tener incluido el componente estacional. A pesar de que este método tiene una búsqueda tipo `GridSearch` integrado, siempre se pueden implementar técnicas propias para encontrar el modelo más óptimo.   

Probamos por ejemplo con la combinación $SARIMA(2,0,0)(7,0,1)12$

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
modelo = SARIMAX(df_train,
                order=(2,0,0),
                seasonal_order=(7,0,1,12))
fit_mod = modelo.fit()
print(fit_mod.summary().tables[0])
estim_mod = fit_mod.fittedvalues
pred_mod = fit_mod.forecast(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_mod))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_mod)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_mod))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_mod)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_mod))



---

### Exponential Smoothing

El suavizado exponencial (__Exponential Smoothing__) es un método para modelar las series de tiempo que se basa en la idea de que **el valor futuro de una serie de tiempo es un promedio ponderado de sus valores pasados**, donde las **observaciones más recientes reciben pesos más altos** que las observaciones más antiguas. El método se llama "`exponencial`" porque **los pesos disminuyen exponencialmente** a medida que las observaciones envejecen.

Este método es lo suficientemente flexible para **manejar diferentes tipos de patrones** en series temporales, como tendencias, estacionalidad e incluso las fluctuaciones irregulares.

Hay distintas variaciones de Suavizado exponencial, cada una de las cuales hace suposiciones diferentes sobre los datos. Las variantes más utilizadas son:

- **Simple Exponential Smoothing**: Este método asume que la serie de tiempo **no tiene tendencia ni estacionalidad**, y el pronóstico para el próximo valor se basa únicamente en el promedio ponderado de las observaciones anteriores.

- **Holt's Linear Exponential Smoothing**: Este método asume que la serie de tiempo **tiene una tendencia lineal** pero no estacionalidad.

- **Holt-Winters Exponential Smoothing**: Este método asume que la serie de tiempo **tiene una tendencia lineal** y **un componente estacional**, y el valor para el próximo paso se basa en un promedio ponderado de observaciones pasadas, un componente de tendencia y un componente estacional.

El suavizado exponencial es un método ampliamente utilizado por su simplicidad, **flexibilidad** y capacidad para capturar diferentes tipos de patrones en series temporales. Sin embargo, tiene algunas limitaciones, como su **incapacidad para manejar patrones muy complejos** y el hecho de que asume que los valores futuros de la serie dependen únicamente de sus valores pasados.

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

modelo_exp = ExponentialSmoothing(df_train,
                                  trend='add',
                                  seasonal='add',
                                  seasonal_periods=12)

fit_exp = modelo_exp.fit()
fit_exp.summary()


In [ ]:
estim_exp = fit_exp.fittedvalues
plt.plot(df_train, label='Entrenamiento')
plt.plot(estim_exp, label='Estimación Exponential Smoothing')
plt.title("Datos reales vs. Estimación del modelo (In-Sample forecasting)")
plt.legend()
plt.show()

In [ ]:
# Comparar los valores reales con la estimación del modelo
sns.scatterplot(x=df_train.values.flatten(), y=estim_exp)
plt.plot([0,600], [0,600], color='r', linestyle=':')
plt.title("Valores reales vs. valors estimados (In-Sample forecasting) Exponential Smoothing")
plt.show()

In [ ]:
# Analizamos el componente residual
resid_exp = fit_exp.resid

# Coeficiente de correlación entre valores reales y los errores
print(df_train['passengers'].corr(resid_exp).round(4))

# Coeficiente de correlación entre valores estimados y los errores
print(estim_exp.corr(resid_exp).round(4))

sns.scatterplot(x=df_train.values.flatten(), y=resid_exp)
plt.title("Valores reales versus valores residuales - Exponential Smoothing")
plt.show()

In [ ]:
sns.scatterplot(x=estim_exp, y=resid_exp)
plt.title("Valores estimados versus valores residuales - Exponential Smoothing")
plt.show()

In [ ]:
pred_exp = fit_exp.forecast(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_exp))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_exp)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_exp))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_exp)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_exp))

Vemos que la precisión de los valores predichos parece aceptable, aunque posiblemente queda información no extraída por el modelo que hace que haya algo de correlación entre valores residuales y los valores reales o estimados por el modelo.

---

### **`Ejercicio 25.1`**

Vamos a analizar los datos de **`Sunspots Dataset`** que son números promediados mensuales de **manchas solares desde 1749 hasta 1983**.  


**`25.1.1`** Utiliza el siguiente enlace para descargar los datos y crear una tabla tipo _DataFrame_ con ellos:
 - 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'

**`25.1.2`** construye la serie temporal, del modo que las fechas tipo `'1749-05-01'` formen los índices, y número de las manchas solares los valores de la serie.


### **`Ejercicio 25.2`**

Vamos a modelar la serie temporal con el objetivo de **predecir los valores mensuales del último año**.

**`25.2.1`** Aplica el método de `Seasonal Naïve` a la serie temporal y consigue estos puntos **y analiza los resultados obtenidos**:
- Grafica de "_Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)_"
- Gráfica interactiva con el histórico, la predicción y los valores reales del último año.
- Las métricas de "_Out-of-sample performance_": MAE, MAPE, MSE, RMSE y R2.

**`25.2.2`** Aplica el método de `Autoregresión - AR(24)` la serie temporal y consigue estos puntos **y analiza los resultados obtenidos**:
- Resumen del modelo ajustado (_Model fit summary_)
- Gráfica de "_Datos reales vs. Estimación del modelo (In-Sample forecasting)_"
- Gráficas de análisis del componente residual (curvas de diagnóstico, correlaciones con otras variables)
- Gráfica de "_Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)_"
- Gráfica interactiva con el histórico, la predicción y los valores reales del último año.
- Las métricas de "_Out-of-sample performance_": MAE, MAPE, MSE, RMSE y R2.

**`25.2.3`** Aplica el método de `SARIMA (4,1,1)(2,0,0)12` la serie temporal y consigue estos puntos **y analiza los resultados obtenidos**:
- Resumen del modelo ajustado (_Model fit summary_)
- Gráfica de "_Datos reales vs. Estimación del modelo (In-Sample forecasting)_"
- Gráficas de análisis del componente residual (curvas de diagnóstico, correlaciones con otras variables)
- Gráfica de "_Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)_"
- Gráfica interactiva con el histórico, la predicción y los valores reales del último año.
- Las métricas de "_Out-of-sample performance_": MAE, MAPE, MSE, RMSE y R2.

**`25.2.4`** Aplica el método de `Holt-Winters Exponential Smoothing` la serie temporal y consigue estos puntos **y analiza los resultados obtenidos**:
- Resumen del modelo ajustado (_Model fit summary_)
- Gráfica de "_Datos reales vs. Estimación del modelo (In-Sample forecasting)_"
- Gráficas de análisis del componente residual (correlaciones con otras variables)
- Gráfica de "_Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)_"
- Gráfica interactiva con el histórico, la predicción y los valores reales del último año.
- Las métricas de "_Out-of-sample performance_": MAE, MAPE, MSE, RMSE y R2.

---